In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data=pd.read_csv('labeled_data.csv')
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
data.describe()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000


In [4]:
data['class'].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [5]:
data1=data[['class','tweet']]
data1

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [6]:
y=data1.loc[:,['class']]

In [7]:
y

,class
0,2
1,1
2,1
3,1
4,1
...,...
24778,1
24779,2
24780,1
24781,1


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [9]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
y = np.array(ct.fit_transform(y))

In [10]:
y

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [11]:
y_hate=np.array(y[:,0])
y_offensive=np.array(y[:,1])

In [12]:
y_hate.shape

(24783,)

In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
corpus = []
for i in range(0, 24783):
  review = re.sub('[^a-zA-Z]', ' ', data1['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [16]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus)

In [17]:
X[0].shape

(1, 2000)

In [18]:
y_hate.shape

(24783,)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y_hate, test_size = 0.20, random_state = 0)

In [20]:
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [21]:

y_pred_rf = classifier_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)
print(cm)

[[4586   92]
 [ 213   66]]


In [22]:
data1['tweet'][1]

'!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!'

In [23]:
print(classification_report(y_pred_rf,y_test))

              precision    recall  f1-score   support

         0.0       0.98      0.96      0.97      4799
         1.0       0.24      0.42      0.30       158

    accuracy                           0.94      4957
   macro avg       0.61      0.69      0.63      4957
weighted avg       0.96      0.94      0.95      4957



In [24]:
print(accuracy_score(y_pred_rf,y_test))

0.9384708493040145


In [25]:
data2=data[['class','tweet']]
data2

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [26]:
data2['NonOffensive']=data2[data2['class']==2]['class']

<ipython-input-26-77c9a7128384>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['NonOffensive']=data2[data2['class']==2]['class']


In [27]:
data2['NonOffensive']=data2['NonOffensive'].apply(lambda x : 1 if x==2.0 else 0)

<ipython-input-27-82a20f0a60b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['NonOffensive']=data2['NonOffensive'].apply(lambda x : 1 if x==2.0 else 0)


In [28]:
data2

,class,tweet,NonOffensive
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,1
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0
...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0
24779,2,"you've gone and broke the wrong heart baby, an...",1
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,0
24781,1,youu got wild bitches tellin you lies,0


In [29]:
data2['offensive']=data2[(data2['class']==0) | (data2['class']==1)]['class']

<ipython-input-29-238286ddd4b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['offensive']=data2[(data2['class']==0) | (data2['class']==1)]['class']


In [30]:
data2

,class,tweet,NonOffensive,offensive
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,1,NaN
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,1.0
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,1.0
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,1.0
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,1.0
...,...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,1.0
24779,2,"you've gone and broke the wrong heart baby, an...",1,NaN
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,0,1.0
24781,1,youu got wild bitches tellin you lies,0,1.0


In [31]:
data2['offensive']=data2['offensive'].apply(lambda x : 1 if(x==0.0 or x==1.0) else 0)

<ipython-input-31-5cfb9c3cc67f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['offensive']=data2['offensive'].apply(lambda x : 1 if(x==0.0 or x==1.0) else 0)


In [32]:
data2

,class,tweet,NonOffensive,offensive
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,1,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,1
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,1
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,1
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,1
...,...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,1
24779,2,"you've gone and broke the wrong heart baby, an...",1,0
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,0,1
24781,1,youu got wild bitches tellin you lies,0,1


In [33]:
data2['target']=data2['offensive'].apply(lambda x : 1 if(x==1) else 0)

<ipython-input-33-6630274ca423>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['target']=data2['offensive'].apply(lambda x : 1 if(x==1) else 0)


In [34]:
data2

,class,tweet,NonOffensive,offensive,target
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,1,0,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,1,1
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,1,1
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,1,1
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,1,1
...,...,...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,1,1
24779,2,"you've gone and broke the wrong heart baby, an...",1,0,0
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,0,1,1
24781,1,youu got wild bitches tellin you lies,0,1,1


In [35]:
corpus = []
for i in range(0, 24783):
  review = re.sub('[^a-zA-Z]', ' ', data2['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)


In [36]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus)

In [37]:
X.shape

(24783, 2000)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, data2['target'], test_size = 0.20, random_state = 0)

In [39]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train,y_train)

SVC()

In [40]:
predictions = model.predict(X_test)

In [41]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       826
           1       0.98      0.97      0.97      4131

    accuracy                           0.96      4957
   macro avg       0.92      0.93      0.92      4957
weighted avg       0.96      0.96      0.96      4957



In [42]:
print(accuracy_score(y_test,predictions))

0.9568287270526528


In [43]:
s_x='hello'

s_x=cv.transform([s_x])
s_x.shape


(1, 2000)

In [44]:
model.predict(s_x)

array([0], dtype=int64)

In [46]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)

In [47]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.835, total= 1.2min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.834, total= 1.1min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.834, total= 1.2min
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.835, total= 1.2min
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.833, total= 1.3min
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.917, total=  14.7s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.917, total=  15.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.906, total=  19.7s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.915, total=  27.9s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.962, total=   8.1s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.954, total=  10.4s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.953, total=  11.9s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.962, total=  12.9s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.960, total=  12.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.957, total=  16.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.951, total=  16.1s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .

[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.954, total=  12.5s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.949, total=  12.8s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.962, total=  11.0s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.953, total=  10.8s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.953, total=  11.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.962, total=  11.8s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.959, total=  12.0s


[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 91.3min finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [48]:
grid.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [49]:
model_grid= SVC(C=10,gamma=0.01)
model_grid.fit(X_train,y_train)
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       826
           1       0.98      0.97      0.97      4131

    accuracy                           0.96      4957
   macro avg       0.92      0.93      0.92      4957
weighted avg       0.96      0.96      0.96      4957



In [50]:
print(accuracy_score(y_test,predictions))

0.9568287270526528


In [51]:
confusion_matrix(y_test,predictions)

array([[ 738,   88],
       [ 126, 4005]], dtype=int64)

In [52]:
prediction_train=model_grid.predict(X_train)

In [53]:
confusion_matrix(y_train,prediction_train)

array([[ 3169,   168],
       [  440, 16049]], dtype=int64)

In [57]:
model_grid.degree

3

<3041x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 20277 stored elements in Compressed Sparse Row format>